In [1]:
import torch

torch.__version__


'2.0.0+cu118'

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)

device= cuda


In [3]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-uncased')
#需要移动到cuda上
pretrained.to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)


#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = torch.nn.Linear(1,16)
        self.linear2 = torch.nn.Linear(1,16)
        self.linear3 = torch.nn.Linear(1,16)
        self.linear4 = torch.nn.Linear(1,16)

        self.fc = torch.nn.Linear(768+3, 2)

    def forward(self, input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
        # print(code_len)
        # code_len = self.linear1(code_len)
        # keyword_num = self.linear2(keyword_num)
        # keyword_fix = self.linear3(keyword_fix)
        # permission = self.linear4(permission)

        # code_len = code_len
        # keyword_num = keyword_num.unsqueeze(1)
        # keyword_fix = keyword_fix.unsqueeze(1)
        # permission = permission.unsqueeze(1)
        # code_len = torch.transpose(code_len, 0, 1)
        # keyword_num = torch.transpose(keyword_num, 0, 1)
        # keyword_fix = torch.transpose(keyword_fix, 0, 1)
        # permission = torch.transpose(permission, 0, 1)


        concat = torch.cat((out.last_hidden_state[:, 0], code_len, keyword_num, permission), dim=1)
        out = self.fc(concat)

        out = out.softmax(dim=1)

        return out


model = Model()
#同样要移动到cuda
model.to(device)

#虚拟一批数据,需要把所有的数据都移动到cuda上
input_ids = torch.ones(16, 100).int().to(device)
attention_mask = torch.ones(16, 100).int().to(device)
token_type_ids = torch.ones(16, 100).int().to(device)
code_len = torch.ones(16,1).float().to(device)
keyword_num = torch.ones(16,1).float().to(device)
permission = torch.ones(16,1).float().to(device)
labels = torch.ones(16,1).float().to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids,
      code_len=code_len,
      keyword_num=keyword_num,
      permission=permission).shape

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002B775C5BDF0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/config.json
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining

torch.Size([16, 2])

In [4]:
from datasets import load_from_disk
from datasets import load_dataset
import random
from sklearn.model_selection import train_test_split

dataset1 = load_dataset(path='csv', data_dir='D:\\hzrproject\\study\\n_feature2',data_files='modified_issueDataAll.csv')['train']

#train_dataset, test_dataset = dataset1.train_test_split(test_size=0.2, seed=42)


train_ratio = 0.8  # 80% 用于训练，20% 用于测试
num_samples = len(dataset1)

# 创建一个索引列表，代表数据集的样本
sample_indices = list(range(num_samples))

# 随机打乱索引列表
random.shuffle(sample_indices)

# 计算分割点
split_index = int(train_ratio * num_samples)

# 使用随机抽样来选择训练集和测试集的索引
train_indices = sample_indices[:split_index]
test_indices = sample_indices[split_index:]

# 使用索引来选择子集
train_dataset = dataset1.select(train_indices)
test_dataset = dataset1.select(test_indices)

print(train_dataset)

#定义数据集
class Dataset:
    def __init__(self):
        self.dataset = train_dataset


    def __len__(self):

        return len(self.dataset)

    def __getitem__(self, i):
        ds = self.dataset[i]['ds']
        code_len = self.dataset[i]['code_len']
        keyword_num = self.dataset[i]['keyword_num']
        keyword_fix = self.dataset[i]['keyword_fix']
        permission = self.dataset[i]['permission']
        label = self.dataset[i]['label']

        return  ds, code_len, keyword_num, keyword_fix, permission, label

class Dataset_validation:
    def __init__(self):
        self.dataset_validation = test_dataset

    def __len__(self):
        return len(self.dataset_validation)

    def __getitem__(self, i):
        ds = self.dataset_validation[i]['ds']
        code_len = self.dataset_validation[i]['code_len']
        keyword_num = self.dataset_validation[i]['keyword_num']
        keyword_fix = self.dataset_validation[i]['keyword_fix']
        permission = self.dataset_validation[i]['permission']
        label = self.dataset_validation[i]['label']

        return  ds, code_len, keyword_num, keyword_fix, permission, label


dataset = Dataset()
dataset_validation = Dataset_validation()

len(dataset), dataset[0], len(dataset_validation), dataset_validation[0]

Found cached dataset csv (C:/Users/hananan/.cache/huggingface/datasets/csv/default-cdeb13bd05363744/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['num', 'label', 'code_len', 'con_num', 'keyword_num', 'keyword_fix', 'username', 'permission', 'ds'],
    num_rows: 1996
})


(1996,
 ('title:Killing docker-containerd breaks interaction with containers. description:When killing `docker-containerd`, interacting with containers (`docker exec`, `docker stop`, `docker kill`) fails:```bashdocker kill testingError response from daemon: Cannot kill container: testing: Cannot kill container 9bfdba3fc8eee79d6ca5773f7caff5dc5a8379037e98b6ded5c8b68df5750359: connection error: desc = "transport: dial unix /var/run/docker/containerd/docker-containerd.sock: connect: connection refused": unknowndocker rm -f lucid_yalowError response from daemon: Could not kill running container 9bfdba3fc8eee79d6ca5773f7caff5dc5a8379037e98b6ded5c8b68df5750359, cannot remove - Cannot kill container 9bfdba3fc8eee79d6ca5773f7caff5dc5a8379037e98b6ded5c8b68df5750359: connection error: desc = "transport: dial unix /var/run/docker/containerd/docker-containerd.sock: connect: connection refused": unknown```But killing `dockerd` (either by `killall -9 dockerd` or a `SIGHUP`; `killall -HUP dockerd`) r

In [5]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-uncased')


def collate_fn(data):
    sents = [i[0] for i in data]
    code_len = [i[1] for i in data]
    keyword_num = [i[2] for i in data]
    permission = [i[4] for i in data]
    labels = [i[5] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)

    code_len = torch.FloatTensor(code_len).to(device)
    code_len = code_len.unsqueeze(0).t()

    keyword_num = torch.FloatTensor(keyword_num).to(device)
    keyword_num = keyword_num.unsqueeze(0).t()

    permission = torch.FloatTensor(permission).to(device)
    permission = permission.unsqueeze(0).t()

    labels = torch.LongTensor(labels).to(device).to(device)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, code_len.shape, keyword_num.shape, permission.shape, labels

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002B8062A4CA0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/added_tokens.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002B77FBB9340>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/special_tokens_map.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection objec

OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [6]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=2e-5)

criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(40):
    all_right = 0
    all_label = 0
    print(f"epoch: {epoch+1}")
    for i, (input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission,
            labels) in enumerate(loader):
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    code_len=code_len,
                    keyword_num=keyword_num,
                    permission=permission)

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:

            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            all_right = all_right + (out == labels).sum().item()
            all_label = all_label + len(labels)
            print(i, loss.item(), accuracy)
        if i == 1000:
            break

    print(all_right/all_label)
    print("----------------------")



D:\python\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch: 1
0 0.9698920249938965 0.25
10 0.9243355393409729 0.3125
20 0.8281766176223755 0.5
30 0.8890995979309082 0.25
40 0.7533174157142639 0.625
50 0.8316081762313843 0.375
60 0.8855512142181396 0.5
70 0.7870410680770874 0.5625
80 0.7157672643661499 0.5625
90 0.7866870164871216 0.3125
100 0.8191074728965759 0.375
110 0.7440036535263062 0.5
120 0.8365598320960999 0.375
0.4230769230769231
----------------------
epoch: 2
0 0.5942975282669067 0.6875
10 0.8554688692092896 0.5
20 0.8203074932098389 0.5
30 0.5759769082069397 0.6875
40 0.8705627918243408 0.375
50 0.7841891646385193 0.5625
60 0.8588594794273376 0.5
70 0.7886132001876831 0.5
80 0.8531505465507507 0.4375
90 0.7988027930259705 0.5
100 0.7362814545631409 0.625
110 0.8017143607139587 0.5
120 0.7278761863708496 0.5
0.5288461538461539
----------------------
epoch: 3
0 0.6085062026977539 0.6875
10 0.699018657207489 0.625
20 0.801583468914032 0.4375
30 0.6989946961402893 0.5625
40 0.6608951091766357 0.6875
50 0.8677161931991577 0.375
60

In [7]:
#测试
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset_validation,
                                              batch_size=50,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    y_true = []
    y_pred = []

    for i, (input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission,
            labels) in enumerate(loader_test):

        if i == 20:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    code_len=code_len,
                    keyword_num=keyword_num,
                    permission=permission)

        out = out.argmax(dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(out.cpu().numpy())

        correct += (out == labels).sum().item()
        total += len(labels)

    # print(y_true)
    # print(y_pred)
    accuracy = correct / total
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, pos_label=0, average='binary')
    auc_score = roc_auc_score(y_true, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
    print(f"AUC score: {auc_score}")


test()

0
1
2
3
4
5
6
7
8
9
Accuracy: 0.622
Precision: 0.6301775147928994
Recall: 0.7689530685920578
F1-score: 0.6926829268292682
AUC score: 0.6042074760000647
